<a href="https://colab.research.google.com/github/CrisEsp/Heur-stico/blob/secundario/Heuristico_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import random
import time
import threading
import sys
import threading
from queue import Queue

# Agregar esto al inicio de tu script, junto con otras variables globales
tipos_produccion_actual = {
    "MC1": None,
    "MC2": None,
    "MC3": None
}

# Cola global para materiales pendientes de alimentación
cola_alimentacion = Queue()


# Definición de capacidades de tolvas y niveles iniciales
tolvas_MC1 = {
    "Clinker": {"capacidad": 500, "max_metros": 14},
    "Puzolana": {"capacidad": 300, "max_metros": 12},
    "Yeso": {"capacidad": 300, "max_metros": 10}
}
tolvas_MC2 = {
    "Clinker": {"capacidad": 300, "max_metros": 9},
    "Puzolana_Humeda": {"capacidad": 500, "max_metros": 15, "tolva": "426HO04"},
    "Puzolana_Seca": {"capacidad": 100, "max_metros": 12, "tolva": "426HO02"},
    "Yeso": {"capacidad": 120, "max_metros": 9}
}
tolvas_MC3 = {
    "Clinker": {"capacidad": 60, "max_porcentaje": 100},
    "Clinker_Silo_Blanco": {"capacidad": 500, "max_metros": 10.5},
    "Puzolana": {"capacidad": 35, "max_porcentaje": 100},
    "Yeso": {"capacidad": 30, "max_porcentaje": 100}
}

niveles_MC1 = {"Clinker": 0, "Puzolana": 0, "Yeso": 0}
niveles_MC2 = {"Clinker": 0, "Puzolana_Humeda": 0, "Puzolana_Seca": 0, "Yeso": 0}
niveles_MC3 = {"Clinker": 0, "Clinker_Silo_Blanco": 0, "Puzolana": 0, "Yeso": 0}

# Variables globales para rastrear alimentaciones y temporizadores
alimentaciones_actuales = set()
alimentaciones_en_progreso = {}
temporizadores_molinos = {}
lock = threading.Lock()
lock_print = threading.Lock()




# Funciones auxiliares
def calcular_tiempo_vaciado(molino, material):
    if molino == "MC1":
        niveles = niveles_MC1
        tolvas = tolvas_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
        tolvas = tolvas_MC2
    else:  # MC3
        niveles = niveles_MC3
        tolvas = tolvas_MC3

    capacidad = tolvas[material]['capacidad']
    nivel_actual = niveles[material]
    nivel_ineficiente = nivel_ineficiente_tolva(molino, material)

    if 'max_metros' in tolvas[material]:
        max_nivel = tolvas[material]['max_metros']
        toneladas_reales = ((nivel_actual - nivel_ineficiente) * capacidad) / max_nivel
    else:  # Para MC3 que usa porcentajes
        toneladas_reales = ((nivel_actual - nivel_ineficiente) * capacidad) / 100

    consumo = calcular_consumo(molino, material)

    if consumo > 0:
        return toneladas_reales / consumo
    else:
        return float('inf')


def nivel_silo_blanco():
    return niveles_MC3["Clinker_Silo_Blanco"] / tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"] * tolvas_MC3["Clinker_Silo_Blanco"]["max_metros"]

def nivel_426HO04():
    return niveles_MC2["Puzolana_Humeda"] / tolvas_MC2["Puzolana_Humeda"]["capacidad"] * tolvas_MC2["Puzolana_Humeda"]["max_metros"]

def nivel_puzolana_humeda():
    return niveles_MC2["Puzolana_Humeda"] / tolvas_MC2["Puzolana_Humeda"]["capacidad"] * tolvas_MC2["Puzolana_Humeda"]["max_metros"]

def se_alimenta_puzolana_a_L1():
    return ("MC1", "Puzolana") in alimentaciones_actuales


def se_alimenta_yeso_a_L1():
    return ("MC1", "Yeso") in alimentaciones_actuales

def se_desea_producir_P10():
    return obtener_tipo_produccion_actual("MC2") == "P10"

def hay_clinker_disponible_en_galpon():
    # Esta función debería verificar si hay clinker disponible en el galpón
    # Por ahora, retornamos un valor aleatorio
    return random.choice([True, False])

def se_alimenta_yeso_a_L1_por_L1():
    return ("MC1", "Yeso") in alimentaciones_actuales and alimentaciones_en_progreso.get("MC1", set()).intersection({"Yeso"})

def se_alimenta_clinker_a_L3():
    return ("MC3", "Clinker") in alimentaciones_actuales

def se_puede_alimentar_L1_por_L2():
    return not any(alim[0] == "MC2" for alim in alimentaciones_actuales)

def se_alimenta_yeso_a_L2_por_L2():
    return ("MC2", "Yeso") in alimentaciones_actuales

def se_alimenta_yeso_a_L3():
    return ("MC3", "Yeso") in alimentaciones_actuales

def se_alimenta_yeso_a_L3_por_L2():
    return ("MC3", "Yeso") in alimentaciones_actuales and ("MC2", "Yeso") in alimentaciones_actuales

def se_alimenta_yeso_a_L1_por_L2():
    return ("MC1", "Yeso") in alimentaciones_actuales and ("MC2", "Yeso") in alimentaciones_actuales

def se_alimenta_puzolana_a_L1_por_L2():
    return ("MC1", "Puzolana") in alimentaciones_actuales and ("MC2", "Puzolana_Humeda") in alimentaciones_actuales

def se_alimenta_PH_a_426HO04_por_L2():
    return ("MC2", "Puzolana_Humeda") in alimentaciones_actuales


def seleccionar_ruta_alimentacion(molino, material):
    if material == "Clinker":
        return seleccionar_ruta_clinker(molino)
    elif material in ["Puzolana", "Puzolana_Humeda", "Puzolana_Seca"]:
        return seleccionar_ruta_puzolana(molino, material)
    elif material == "Yeso":
        return seleccionar_ruta_yeso(molino)
    else:
        return "Material no reconocido"

def seleccionar_ruta_clinker(molino):
    print(f"\n--- Seleccionando ruta de Clinker para {molino} ---")

    tiempo_vaciado_L1 = calcular_tiempo_vaciado("MC1", "Clinker")
    tiempo_vaciado_L2 = calcular_tiempo_vaciado("MC2", "Clinker")
    tiempo_vaciado_L3 = calcular_tiempo_vaciado("MC3", "Clinker")

    print(f"Tiempos de vaciado: L1={tiempo_vaciado_L1:.2f}h, L2={tiempo_vaciado_L2:.2f}h, L3={tiempo_vaciado_L3:.2f}h")

    if tiempo_vaciado_L1 > tiempo_vaciado_L2 and tiempo_vaciado_L1 > tiempo_vaciado_L3:
        print("L1 tiene el mayor tiempo de vaciado")
        if not se_alimenta_puzolana_a_L1() and not se_alimenta_yeso_a_L1():
            print("No se está alimentando Puzolana ni Yeso a L1")
            return "Hacia MC1 desde Pretrit"
        else:
            print("Se está alimentando Puzolana o Yeso a L1, no se puede alimentar Clinker")
    elif tiempo_vaciado_L2 > tiempo_vaciado_L3:
        print("L2 tiene el segundo mayor tiempo de vaciado")
        if se_desea_producir_P10():
            print("Se desea producir P10")
            if hay_clinker_disponible_en_galpon():
                print("Hay Clinker disponible en el galpón")
                return "Hacia MC2 desde Pretrit"
            else:
                print("No hay Clinker disponible en el galpón")
                return "Alimentar clinker a galpón"
        else:
            print("No se desea producir P10")
            return "Hacia MC2 desde Pretrit"
    else:
        print("L3 tiene el menor tiempo de vaciado o es igual a L2")
        nivel_actual = nivel_silo_blanco()
        print(f"Nivel actual del Silo Blanco: {nivel_actual:.2f}m")
        if nivel_actual > 3:
            print("Nivel del Silo Blanco > 3m, alimentando desde Silo3")
            return "Hacia MC3 desde Silo3"
        else:
            print("Nivel del Silo Blanco <= 3m, alimentando Silo 3 desde Pretrit")
            return "Hacia Silo 3 desde Pretrit"

    print("No se encontró una ruta válida para alimentar Clinker")
    return "No se puede alimentar Clinker en este momento"

def seleccionar_ruta_puzolana(molino, tipo_puzolana):
    if tipo_puzolana in ["Puzolana", "Puzolana_Humeda"]:
        tiempo_vaciado_L1 = calcular_tiempo_vaciado("MC1", "Puzolana")
        tiempo_vaciado_L2 = calcular_tiempo_vaciado("MC2", "Puzolana_Humeda")

        if tiempo_vaciado_L1 > tiempo_vaciado_L2:
            if nivel_426HO04() <= 9:
                if not se_alimenta_yeso_a_L1_por_L1() and not se_alimenta_clinker_a_L3():
                    return "Húmeda a MC1 por MC1"
            else:
                if se_puede_alimentar_L1_por_L2():
                    if not se_alimenta_yeso_a_L2_por_L2() and not se_alimenta_yeso_a_L3_por_L2() and \
                       not se_alimenta_yeso_a_L1_por_L2() and not se_alimenta_puzolana_a_L1_por_L2():
                        if not se_alimenta_PH_a_426HO04_por_L2():
                            return "P.H. a 426HO04 por MC2"
        else:
            if not se_alimenta_yeso_a_L2_por_L2() and not se_alimenta_yeso_a_L3_por_L2() and \
               not se_alimenta_yeso_a_L1_por_L2() and not se_alimenta_puzolana_a_L1_por_L2():
                if not se_alimenta_PH_a_426HO04_por_L2():
                    return "Húmeda a MC1 por MC2"

    elif tipo_puzolana == "Puzolana_Seca":
        tiempo_vaciado_PS_L2 = calcular_tiempo_vaciado("MC2", "Puzolana_Seca")
        tiempo_vaciado_PS_L3 = calcular_tiempo_vaciado("MC3", "Puzolana")

        if tiempo_vaciado_PS_L2 > tiempo_vaciado_PS_L3:
            if not se_alimenta_clinker_a_L3() and not se_alimenta_yeso_a_L3():
                ajustar_velocidad_transportador(1100, 1150)
                mostrar_advertencia_contaminacion()
                esperar_vaciado_bandas(3)
                return "P.S a 426HO02 por 426HO04"
        else:
            if not se_alimenta_clinker_a_L3() and not se_alimenta_yeso_a_L3():
                ajustar_velocidad_transportador(1100, 1150)
                mostrar_advertencia_contaminacion()
                esperar_vaciado_bandas(3)
                return "P. seca a MC3 por MC2"

    return "No se puede alimentar puzolana en este momento"


def seleccionar_ruta_yeso(molino):
    print(f"\n--- Seleccionando ruta para Yeso en {molino} ---")

    if molino == "MC1":
        print("Verificando condiciones para MC1...")
        if se_puede_alimentar_por_L1():
            print("Se puede alimentar por L1")
            if puede_alimentar_yeso_a_L1():
                print("Se puede alimentar Yeso a L1")
                if not se_alimenta_puzolana_a_L1():
                    print("No se está alimentando Puzolana a L1")
                    return "Hacia MC1 por MC1"
                else:
                    print("Puzolana siendo alimentada a L1. Yeso en espera.")
                    return "En espera para MC1 por MC1"
            else:
                print("No se puede alimentar Yeso a L1")
        elif se_puede_alimentar_L1_por_L2():
            print("Se puede alimentar L1 por L2")
            if not se_alimenta_puzolana():
                print("No se está alimentando Puzolana")
                return "Hacia MC1 por MC2"
            else:
                print("Se está alimentando Puzolana, no se puede alimentar Yeso por L2")
    elif molino == "MC2":
        print("Verificando condiciones para MC2...")
        if not se_alimenta_puzolana():
            print("No se está alimentando Puzolana")
            return "Hacia MC2 por MC2"
        else:
            print("Se está alimentando Puzolana, no se puede alimentar Yeso")
    elif molino == "MC3":
        print("Verificando condiciones para MC3...")
        if se_puede_alimentar_por_L1():
            print("Se puede alimentar por L1")
            if not se_alimenta_clinker_a_L3():
                print("No se está alimentando Clinker a L3")
                return "Hacia MC3 por MC1"
            else:
                print("Se está alimentando Clinker a L3, no se puede alimentar Yeso por L1")
        elif not se_alimenta_puzolana():
            print("No se está alimentando Puzolana")
            return "Hacia MC3 por MC2"
        else:
            print("Se está alimentando Puzolana, no se puede alimentar Yeso por L2")

    print("No se encontró una ruta válida para alimentar Yeso")
    return "No se puede alimentar Yeso en este momento"

def nivel_ineficiente_tolva(molino, material):
    if molino in ["MC1", "MC2"]:
        return 0.2  # 20% de la capacidad
    else:  # MC3
        return 0.5  # 50% de la capacidad

def calcular_consumo(molino, material):
    tipo_produccion = obtener_tipo_produccion_actual(molino)
    necesidades = calcular_necesidades(molino, tipo_produccion)
    return necesidades.get(material, 0)

def obtener_tipo_produccion_actual(molino):
    # Esta función debería obtener el tipo de producción actual del molino
    # Por ahora, usaremos un valor predeterminado
    if molino == "MC1":
        return "P30"
    elif molino == "MC2":
        return "P16"
    else:  # MC3
        return "P30"

def calcular_necesidades(molino, tipo_produccion):
    produccion_MC1 = {
        "P30": (0.30, 0.015, 72),
        "P40": (0.40, 0.015, 64)
    }
    produccion_MC2 = {
        "P10": (0.10, 0.03, 70),
        "P16": (0.16, 0.025, 80),
        "P20": (0.12, 0.025, 87),
        "P30": (0.30, 0.02, 110)
    }
    produccion_MC3 = {
        "P30": (0.30, 0.025, 37)
    }

    if molino == "MC1" and tipo_produccion in produccion_MC1:
        puzolana, yeso, produccion = produccion_MC1[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana": puzolana * produccion,
            "Yeso": yeso * produccion
        }
    elif molino == "MC2" and tipo_produccion in produccion_MC2:
        puzolana, yeso, produccion = produccion_MC2[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana_Humeda": puzolana * produccion * 0.7,
            "Puzolana_Seca": puzolana * produccion * 0.3,
            "Yeso": yeso * produccion
        }
    elif molino == "MC3" and tipo_produccion in produccion_MC3:
        puzolana, yeso, produccion = produccion_MC3[tipo_produccion]
        clinker = 1 - puzolana - yeso
        return {
            "Clinker": clinker * produccion,
            "Puzolana": puzolana * produccion,
            "Yeso": yeso * produccion
        }

    print(f"Advertencia: No se encontró configuración para {molino} con producto {tipo_produccion}")
    return {"Clinker": 60, "Puzolana": 30, "Yeso": 10}

def requiere_alimentacion_puzolana_humeda(molino):
    if molino == "MC1":
        return niveles_MC1["Puzolana"] < tolvas_MC1["Puzolana"]["capacidad"] * 0.8
    elif molino == "MC2":
        return niveles_MC2["Puzolana_Humeda"] < tolvas_MC2["Puzolana_Humeda"]["capacidad"] * 0.8
    return False

def requiere_alimentacion_puzolana_seca(molino):
    if molino == "MC2":
        return niveles_MC2["Puzolana_Seca"] < tolvas_MC2["Puzolana_Seca"]["capacidad"] * 0.8
    elif molino == "MC3":
        return niveles_MC3["Puzolana"] < tolvas_MC3["Puzolana"]["capacidad"] * 0.5
    return False

def ajustar_velocidad_transportador(min_vel, max_vel):
    velocidad = random.uniform(min_vel, max_vel)
    print(f"Ajustando velocidad del transportador a {velocidad:.2f} RPM")
    return velocidad

def mostrar_advertencia_contaminacion():
    print("ADVERTENCIA: Posible contaminación. Proceda con precaución.")

def esperar_vaciado_bandas(minutos):
    print(f"Esperando {minutos} minutos para vaciado de bandas...")
    time.sleep(minutos)

def se_puede_alimentar_por_L1():
    return not any(alim[0] == "MC1" for alim in alimentaciones_actuales)

def puede_alimentar_yeso_a_L1():
    nivel_actual = niveles_MC1["Yeso"]
    max_nivel = min(tolvas_MC1["Yeso"]["max_metros"], 2)  # Limitar a 2m para yeso en MC1
    return nivel_actual < max_nivel

def se_alimenta_puzolana():
    return any([
        ("MC1", "Puzolana") in alimentaciones_actuales,
        ("MC2", "Puzolana_Humeda") in alimentaciones_actuales,
        ("MC2", "Puzolana_Seca") in alimentaciones_actuales,
        ("MC3", "Puzolana") in alimentaciones_actuales
    ])


def esperar_y_alimentar(molino, material, cantidad):
    while se_alimenta_puzolana_a_L1():
        time.sleep(1)  # Espera 1 segundo antes de volver a verificar

    print(f"\nIntentando nuevamente alimentar {material} a {molino}")
    intentar_alimentar(molino, material, cantidad)


def intentar_alimentar(molino, material, cantidad):
    print(f"\n--- Intentando alimentar {material} a {molino} ---")
    print(f"Cantidad solicitada: {cantidad:.2f}t")

    # Verificar nuevamente si es necesario alimentar
    if molino == "MC1":
        niveles = niveles_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
    else:  # MC3
        niveles = niveles_MC3

    nivel_actual = niveles[material]
    tipo_produccion = tipos_produccion_actual[molino]

    if tipo_produccion is None:
        print(f"No se ha definido un tipo de producción para {molino}. No se puede alimentar {material}.")
        return 0

    necesidades = calcular_necesidades(molino, tipo_produccion)
    cantidad_necesaria = necesidades[material]

    if nivel_actual >= cantidad_necesaria:
        print("----12345---")
        print(f"Ya no es necesario alimentar {material} en {molino}. Nivel actual: {nivel_actual:.2f}t, Necesario: {cantidad_necesaria:.2f}t")
        return 0



# def intentar_alimentar(molino, material, cantidad):
#     print(f"\n--- Intentando alimentar {material} a {molino} ---")
#     print(f"Cantidad solicitada: {cantidad:.2f}t")

#     # Verificar nuevamente si es necesario alimentar
#     if molino == "MC1":
#         niveles = niveles_MC1
#     elif molino == "MC2":
#         niveles = niveles_MC2
#     else:  # MC3
#         niveles = niveles_MC3

#     nivel_actual = niveles[material]
#     tipo_produccion = tipos_produccion_actual[molino]
#     necesidades = calcular_necesidades(molino, tipo_produccion)
#     cantidad_necesaria = necesidades[material]

#     if nivel_actual >= cantidad_necesaria:
#         print(f"Ya no es necesario alimentar {material} en {molino}. Nivel actual: {nivel_actual:.2f}t, Necesario: {cantidad_necesaria:.2f}t")
#         return 0

    ruta = seleccionar_ruta_alimentacion(molino, material)
    print(f"Ruta seleccionada: {ruta}")

    if ruta == "En espera para MC1 por MC1":
        print(f"Alimentación de {material} a {molino} en espera. Se intentará nuevamente cuando termine la alimentación de Puzolana.")
        threading.Thread(target=esperar_y_alimentar, args=(molino, material, cantidad)).start()
        return 0

    if ruta == "No se puede alimentar Yeso en este momento" or ruta == "Material no reconocido":
        print(f"Error: No se puede alimentar {material} a {molino}: {ruta}")
        return 0

    print("Verificando restricciones...")
    validacion, mensaje = validar_restricciones(molino, material)
    if not validacion:
        print(f"No se puede alimentar debido a restricciones: {mensaje}")
        return 0

    with lock:
        print("Agregando alimentación a las listas de control...")
        alimentaciones_actuales.add((molino, material))
        if molino not in alimentaciones_en_progreso:
            alimentaciones_en_progreso[molino] = set()
        alimentaciones_en_progreso[molino].add(material)

    if molino == "MC1":
        niveles = niveles_MC1
        tolvas = tolvas_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
        tolvas = tolvas_MC2
    elif molino == "MC3":
        niveles = niveles_MC3
        tolvas = tolvas_MC3

    print(f"Ajustando velocidad del transportador para {material}...")
    if material in ["Puzolana", "Puzolana_Humeda", "Puzolana_Seca"]:
        velocidad = ajustar_velocidad_transportador(1100, 1150)
    elif material == "Yeso":
        velocidad = ajustar_velocidad_transportador(850, 900)
    else:
        velocidad = "No ajustada"
    print(f"Velocidad del transportador: {velocidad}")

    capacidad = tolvas[material]['capacidad']
    print(f"Capacidad de la tolva: {capacidad}t")

    if 'max_metros' in tolvas[material]:
        max_nivel = tolvas[material]['max_metros']
        nivel_actual = niveles[material] / capacidad * max_nivel
        print(f"Nivel actual: {nivel_actual:.2f}m / {max_nivel}m")
        if molino == "MC1" and material == "Yeso":
            max_nivel = min(max_nivel, 2)  # Limitar a 2m para yeso en MC1
            print(f"Nivel máximo ajustado para Yeso en MC1: {max_nivel}m")
        espacio_disponible = max_nivel - nivel_actual
        cantidad_metros = min(cantidad / capacidad * max_nivel, espacio_disponible)
        cantidad_alimentada = cantidad_metros / max_nivel * capacidad
        print(f"Espacio disponible: {espacio_disponible:.2f}m")
        print(f"Cantidad a alimentar en metros: {cantidad_metros:.2f}m")
    else:  # Para MC3 que usa porcentajes
        max_nivel = tolvas[material]['max_porcentaje']
        nivel_actual = niveles[material] / capacidad * 100
        print(f"Nivel actual: {nivel_actual:.2f}% / {max_nivel}%")
        espacio_disponible = max_nivel - nivel_actual
        cantidad_porcentaje = min(cantidad / capacidad * 100, espacio_disponible)
        cantidad_alimentada = cantidad_porcentaje / 100 * capacidad
        print(f"Espacio disponible: {espacio_disponible:.2f}%")
        print(f"Cantidad a alimentar en porcentaje: {cantidad_porcentaje:.2f}%")

    print(f"Cantidad final a alimentar: {cantidad_alimentada:.2f}t")

    if cantidad_alimentada > 0:
        print(f"Iniciando alimentación de {cantidad_alimentada:.2f}t de {material} a {molino}")
        iniciar_alimentacion(molino, material, cantidad_alimentada)

        print(f"\nEstado actual de las tolvas:")
        print(f"MC1: {niveles_MC1}")
        print(f"MC2: {niveles_MC2}")
        print(f"MC3: {niveles_MC3}")
        print(f"Nivel Silo Blanco: {nivel_silo_blanco():.2f}m")

    else:
        print(f"No se pudo alimentar {material} a {molino}. Tolva llena o se alcanzó el límite máximo.")
        with lock:
            alimentaciones_actuales.discard((molino, material))
            if molino in alimentaciones_en_progreso:
                alimentaciones_en_progreso[molino].discard(material)

    return cantidad_alimentada

def validar_restricciones(molino, material):
    with lock:
        if molino in alimentaciones_en_progreso and material in alimentaciones_en_progreso[molino]:
            return False, f"Ya se está alimentando {material} a {molino}."

        if material == "Clinker" or material == "Clinker_Silo_Blanco":
            clinker_alimentaciones = [alim for alim in alimentaciones_actuales if alim[1] in ["Clinker", "Clinker_Silo_Blanco"]]

            if not clinker_alimentaciones:
                return True, "Validación exitosa"

            if molino == "MC3" or (molino in ["MC1", "MC2"] and any(alim[0] == "MC3" for alim in clinker_alimentaciones)):
                return True, "Validación exitosa"

            if material == "Clinker_Silo_Blanco" and any(alim[0] == "MC3" and alim[1] == "Clinker" for alim in clinker_alimentaciones):
                return True, "Validación exitosa"

            return False, "No se permite esta combinación de alimentación de Clinker"

        if molino == "MC3":
            materiales_actuales = [alim[1] for alim in alimentaciones_actuales if alim[0] == "MC3"]
            if len(materiales_actuales) >= 1 and material not in materiales_actuales:
                return False, f"No se puede alimentar {material} mientras se alimenta {materiales_actuales[0]}"

        if molino == "MC1":
            if material in ["Puzolana", "Yeso"] and "Puzolana" in alimentaciones_en_progreso.get(molino, []) and "Yeso" in alimentaciones_en_progreso.get(molino, []):
                return False, "No se puede alimentar Puzolana y Yeso simultáneamente a MC1"

        if molino == "MC1" and material == "Yeso":
            if niveles_MC1["Yeso"] + 2 > tolvas_MC1["Yeso"]["max_metros"]:
                return False, "No se puede alimentar más Yeso a MC1, se excedería el límite de 2m"

    return True, "Validación exitosa"

def iniciar_alimentacion(molino, material, cantidad):
    if molino not in alimentaciones_en_progreso:
        alimentaciones_en_progreso[molino] = set()
    alimentaciones_en_progreso[molino].add(material)
    temporizadores_molinos[molino] = True
    thread = threading.Thread(target=simular_alimentacion, args=(molino, material, cantidad, 15))
    thread.start()
    print(f"\nIniciando alimentación de {material} a {molino}")

def simular_alimentacion(molino, material, cantidad, duracion_real):
    tiempo_inicio = time.time()
    tiempo_simulado = 0
    while tiempo_simulado < 60:  # 1 hora simulada
        tiempo_actual = time.time() - tiempo_inicio
        tiempo_simulado = (tiempo_actual / duracion_real) * 60
        porcentaje = min(tiempo_simulado / 60 * 100, 100)
        imprimir_progreso(molino, material, porcentaje)
        time.sleep(0.1)

    with lock_print:
        print(f"\n{molino} - Alimentación de {material} completada.")

    # Actualizar niveles después de la alimentación
    with lock:
        if molino == "MC1":
            niveles_MC1[material] += cantidad
        elif molino == "MC2":
            niveles_MC2[material] += cantidad
        elif molino == "MC3":
            niveles_MC3[material] += cantidad


        if molino in alimentaciones_en_progreso:
            alimentaciones_en_progreso[molino].remove(material)
        if (molino, material) in alimentaciones_actuales:
            alimentaciones_actuales.remove((molino, material))

        print(f"\n{molino} - Alimentación de {material} completada. Ruta habilitada para nueva alimentación.")

        print(f"\nEstado actual de las tolvas antes de intentar alimentar ------5:")
        print(f"MC1: {niveles_MC1}")
        print(f"MC2: {niveles_MC2}")
        print(f"MC3: {niveles_MC3}")
        print(f"Nivel Silo Blanco: {nivel_silo_blanco():.2f}m")

    if not alimentaciones_en_progreso.get(molino):
        temporizadores_molinos[molino] = False

def imprimir_progreso(molino, material, porcentaje):
    barra = f"[{'#' * int(porcentaje / 2)}{' ' * (50 - int(porcentaje / 2))}]"
    with lock_print:
        sys.stdout.write(f"\r{molino} - Alimentando {material}: {barra} {porcentaje:.1f}%")
        sys.stdout.flush()


# def optimizar_alimentacion(molino, tipo_produccion):
#     necesidades = calcular_necesidades(molino, tipo_produccion)
#     if molino == "MC1":
#         niveles = niveles_MC1
#     elif molino == "MC2":
#         niveles = niveles_MC2
#     else:  # MC3
#         niveles = niveles_MC3

#     for material, cantidad_necesaria in necesidades.items():
#         nivel_actual = niveles[material]
#         if nivel_actual < cantidad_necesaria:
#             cantidad_faltante = cantidad_necesaria - nivel_actual
#             print(f"Falta {cantidad_faltante:.2f}t de {material} en {molino}")
#             cola_alimentacion.put((molino, material, cantidad_faltante))
#         else:
#             print(f"No es necesario alimentar más {material} en {molino}")

#     print(f"Optimización para {molino} completada.")

#     imprimir_tiempos(molino, tipo_produccion)

#     recomendaciones, restricciones = generar_recomendaciones(molino)
#     print("\nRecomendaciones:")
#     for recomendacion in recomendaciones:
#         print(recomendacion)

#     materiales_alimentados = []
#     for material, cantidad_necesaria in necesidades.items():
#         if cantidad_necesaria > 0:
#             ruta = seleccionar_ruta_alimentacion(molino, material)
#             print(f"\nIntentando alimentar {material} a {molino} por la ruta: {ruta}")
#             cantidad_alimentada = intentar_alimentar(molino, material, cantidad_necesaria)
#             if cantidad_alimentada > 0:
#                 materiales_alimentados.append(material)
#             else:
#                 print(f"No se pudo alimentar {material}. Se intentará en el próximo ciclo.")

#     print("\nRestricciones aplicables:")
#     restricciones_aplicables = [r for r in restricciones if any(mat in r for mat in materiales_alimentados)]
#     for restriccion in restricciones_aplicables:
#         print(restriccion)

#     print("\nTiempos actualizados después de la alimentación:")
#     imprimir_tiempos(molino, tipo_produccion)

#     if molino == "MC1":
#         niveles = niveles_MC1
#     elif molino == "MC2":
#         niveles = niveles_MC2
#     else:  # MC3
#         niveles = niveles_MC3

#     materias_faltantes = verificar_alimentacion_completa(molino, tipo_produccion, niveles)
#     if materias_faltantes:
#         print(f"\nAdvertencia: No se pudo alimentar todas las materias primas necesarias para {molino}.")
#         print(f"Materias primas faltantes: {', '.join(materias_faltantes)}")
#         print("No se puede elaborar el producto con la configuración actual.")
#     else:
#         print(f"\nTodas las materias primas necesarias para {molino} han sido alimentadas correctamente.")
#         print("El producto puede ser elaborado con la configuración actual.")

def optimizar_alimentacion(molino, tipo_produccion):
    necesidades = calcular_necesidades(molino, tipo_produccion)
    if molino == "MC1":
        niveles = niveles_MC1
    elif molino == "MC2":
        niveles = niveles_MC2
    else:  # MC3
        niveles = niveles_MC3

    print(f"\nOptimizando alimentación para {molino} - Tipo de producción: {tipo_produccion}")
    print("Necesidades calculadas:")
    for material, cantidad in necesidades.items():
        print(f"  {material}: {cantidad:.2f}t")
    print("Niveles actuales:")
    for material, nivel in niveles.items():
        print(f"  {material}: {nivel:.2f}t")

    # materias_faltantes = verificar_alimentacion_completa(molino, tipo_produccion, niveles)

    # if not materias_faltantes:
    #     print(f"Todas las materias primas necesarias para {molino} están en niveles adecuados.")
    #     return

    # for material in materias_faltantes:
    #     cantidad_necesaria = necesidades[material]
    #     nivel_actual = niveles[material]
    #     cantidad_faltante = cantidad_necesaria - nivel_actual
    #     if cantidad_faltante > 0:
    #         print(f"Falta {cantidad_faltante:.2f}t de {material} en {molino}")
    #         cola_alimentacion.put((molino, material, cantidad_faltante))
    #     else:
    #         print(f"No es necesario alimentar más {material} en {molino}")

    necesidades = calcular_necesidades(molino, tipo_produccion)
    materias_faltantes = verificar_alimentacion_completa(molino, tipo_produccion, niveles)

    if not materias_faltantes:
        print(f"Todas las materias primas necesarias para {molino} están en niveles adecuados.")
        return

    for material in materias_faltantes:
        cantidad_necesaria = necesidades[material]
        nivel_actual = niveles[material]
        cantidad_faltante = cantidad_necesaria - nivel_actual
        if cantidad_faltante > 0:
            print(f"Falta {cantidad_faltante:.2f}t de {material} en {molino}")
            cola_alimentacion.put((molino, material, cantidad_faltante))
        else:
            print(f"No es necesario alimentar más {material} en {molino}")

    print(f"Optimización para {molino} completada.")

    imprimir_tiempos(molino, tipo_produccion)

    recomendaciones, restricciones = generar_recomendaciones(molino)
    print("\nRecomendaciones:")
    for recomendacion in recomendaciones:
        print(recomendacion)

    materiales_alimentados = []
    for material in materias_faltantes:
        ruta = seleccionar_ruta_alimentacion(molino, material)
        print(f"\nIntentando alimentar {material} a {molino} por la ruta: {ruta}")
        cantidad_alimentada = intentar_alimentar(molino, material, necesidades[material])
        if cantidad_alimentada > 0:
            materiales_alimentados.append(material)
        else:
            print(f"No se pudo alimentar {material}. Se intentará en el próximo ciclo.")

    print("\nRestricciones aplicables:")
    restricciones_aplicables = [r for r in restricciones if any(mat in r for mat in materiales_alimentados)]
    for restriccion in restricciones_aplicables:
        print(restriccion)

    print("\nTiempos actualizados después de la alimentación:")
    imprimir_tiempos(molino, tipo_produccion)

    materias_faltantes = verificar_alimentacion_completa(molino, tipo_produccion, niveles)
    if materias_faltantes:
        print(f"\nAdvertencia: No se pudo alimentar todas las materias primas necesarias para {molino}.")
        print(f"Materias primas faltantes: {', '.join(materias_faltantes)}")
        print("No se puede elaborar el producto con la configuración actual.")
    else:
        print(f"\nTodas las materias primas necesarias para {molino} han sido alimentadas correctamente.")
        print("El producto puede ser elaborado con la configuración actual.")


def imprimir_tiempos(molino, tipo_produccion):
    necesidades = calcular_necesidades(molino, tipo_produccion)
    tiempos_llenado = calcular_tiempos_llenado(molino, necesidades)

    print(f"\nTiempos de llenado y vaciado para {molino} - {tipo_produccion}:")
    print("-" * 60)
    print(f"{'Material':<15}{'Tiempo Llenado (h)':<20}{'Tiempo Vaciado (h)':<20}")
    print("-" * 60)

    for material, tiempo_llenado in tiempos_llenado.items():
        tiempo_vaciado = calcular_tiempo_vaciado(molino, material)
        tiempo_llenado_str = f"{tiempo_llenado:.2f}" if tiempo_llenado != float('inf') else "∞"
        tiempo_vaciado_str = f"{tiempo_vaciado:.2f}" if tiempo_vaciado != float('inf') else "∞"
        print(f"{material:<15}{tiempo_llenado_str:<20}{tiempo_vaciado_str:<20}")

def calcular_tiempos_llenado(molino, necesidades):
    tiempos_llenado = {}
    if molino == "MC1":
        rendimiento = 0.8
        for mat, cant in necesidades.items():
            if mat == "Yeso":
                capacidad_objetivo = min(tolvas_MC1[mat]["capacidad"] * rendimiento, 2 / tolvas_MC1[mat]["max_metros"] * tolvas_MC1[mat]["capacidad"])
            else:
                capacidad_objetivo = tolvas_MC1[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC1[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')

    elif molino == "MC2":
        rendimiento = 0.8
        for mat, cant in necesidades.items():
            capacidad_objetivo = tolvas_MC2[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC2[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')
    else:  # MC3
        rendimiento = 0.5
        for mat, cant in necesidades.items():
            capacidad_objetivo = tolvas_MC3[mat]["capacidad"] * rendimiento
            nivel_actual = niveles_MC3[mat]
            if cant > 0:
                tiempos_llenado[mat] = (capacidad_objetivo - nivel_actual) / cant
            else:
                tiempos_llenado[mat] = float('inf')

    return tiempos_llenado

def generar_recomendaciones(molino_actual):
    recomendaciones = []
    restricciones = []

    def espacio_disponible(nivel_actual, capacidad, rendimiento):
        return capacidad * rendimiento - nivel_actual

    print(f"\n--- Datos para generar recomendaciones de {molino_actual} ---")

    if molino_actual == "MC1":
        rendimiento = 0.8
        print(f"Rendimiento de MC1: {rendimiento}")
        for material in ["Clinker", "Puzolana", "Yeso"]:
            nivel_actual = niveles_MC1[material]
            capacidad = tolvas_MC1[material]["capacidad"]
            esp = espacio_disponible(nivel_actual, capacidad, rendimiento)
            print(f"{material}: Nivel actual = {nivel_actual:.2f}t, Capacidad = {capacidad}t, Espacio disponible = {esp:.2f}t")

            if material == "Yeso":
                esp = min(esp, 60 - nivel_actual)  # Limitar a 2m para yeso en MC1
                print(f"Yeso: Espacio disponible ajustado = {esp:.2f}t (limitado a 2m)")

            if esp > 0:
                recomendaciones.append(f"Recomiendo alimentar {material} a MC1 (Espacio disponible: {esp:.2f}t)")

    elif molino_actual == "MC2":
        rendimiento = 0.8
        print(f"Rendimiento de MC2: {rendimiento}")
        for material in ["Clinker", "Puzolana_Humeda", "Puzolana_Seca", "Yeso"]:
            nivel_actual = niveles_MC2[material]
            capacidad = tolvas_MC2[material]["capacidad"]
            esp = espacio_disponible(nivel_actual, capacidad, rendimiento)
            print(f"{material}: Nivel actual = {nivel_actual:.2f}t, Capacidad = {capacidad}t, Espacio disponible = {esp:.2f}t")

            if esp > 0:
                if material == "Clinker":
                    recomendaciones.append(f"Recomiendo alimentar Clinker a MC2 desde Pretrit (Espacio disponible: {esp:.2f}t)")
                elif material == "Puzolana_Humeda":
                    recomendaciones.append(f"Recomiendo alimentar Puzolana Húmeda a 426HO04 por MC2 (Espacio disponible: {esp:.2f}t)")
                elif material == "Puzolana_Seca":
                    recomendaciones.append(f"Recomiendo alimentar Puzolana Seca a 426HO02 por 426HO04 (Espacio disponible: {esp:.2f}t)")
                elif material == "Yeso":
                    recomendaciones.append(f"Recomiendo alimentar Yeso a MC2 por MC2 (Espacio disponible: {esp:.2f}t)")

    elif molino_actual == "MC3":
        rendimiento = 0.5
        print(f"Rendimiento de MC3: {rendimiento}")
        for material in ["Clinker", "Clinker_Silo_Blanco", "Puzolana", "Yeso"]:
            nivel_actual = niveles_MC3[material]
            capacidad = tolvas_MC3[material]["capacidad"]
            esp = espacio_disponible(nivel_actual, capacidad, rendimiento)
            print(f"{material}: Nivel actual = {nivel_actual:.2f}t, Capacidad = {capacidad}t, Espacio disponible = {esp:.2f}t")

            if esp > 0:
                if material == "Clinker":
                    nivel_silo_blanco = niveles_MC3["Clinker_Silo_Blanco"]
                    capacidad_silo_blanco = tolvas_MC3["Clinker_Silo_Blanco"]["capacidad"]
                    porcentaje_silo_blanco = nivel_silo_blanco / capacidad_silo_blanco
                    print(f"Porcentaje de llenado del Silo Blanco: {porcentaje_silo_blanco:.2%}")
                    if porcentaje_silo_blanco > 0.3:
                        recomendaciones.append(f"Recomiendo alimentar Clinker a MC3 desde Silo Blanco (Espacio disponible: {esp:.2f}t)")
                    else:
                        esp_silo_blanco = espacio_disponible(nivel_silo_blanco, capacidad_silo_blanco, rendimiento)
                        recomendaciones.append(f"Recomiendo alimentar Clinker a Silo Blanco desde Pretrit (Espacio disponible: {esp_silo_blanco:.2f}t)")
                elif material == "Puzolana":
                    recomendaciones.append(f"Recomiendo alimentar Puzolana Seca a MC3 por MC2 (Espacio disponible: {esp:.2f}t)")
                elif material == "Yeso":
                    recomendaciones.append(f"Recomiendo alimentar Yeso a MC3 (Espacio disponible: {esp:.2f}t)")

    if not recomendaciones:
        recomendaciones.append(f"Todas las tolvas de {molino_actual} están llenas o cerca de su capacidad máxima.")

    restricciones.append("Es necesario vaciar el sistema antes de una nueva alimentación para evitar contaminación.")
    restricciones.append("No se puede alimentar simultáneamente Clinker, Puzolana y Yeso a MC3.")
    restricciones.append("No se puede alimentar Clinker a múltiples molinos simultáneamente.")

    return recomendaciones, restricciones

def verificar_alimentacion_completa(molino, tipo_produccion, niveles):
    necesidades = calcular_necesidades(molino, tipo_produccion)
    materias_faltantes = []

    for material, cantidad_necesaria in necesidades.items():
        if cantidad_necesaria > 0 and niveles[material] == 0:
            materias_faltantes.append(material)

    return materias_faltantes


# def verificar_alimentacion_completa(molino, tipo_produccion, niveles):
#     necesidades = calcular_necesidades(molino, tipo_produccion)
#     materias_faltantes = []
#     rendimiento = 0.8 if molino in ["MC1", "MC2"] else 0.5

#     for material, cantidad_necesaria in necesidades.items():
#         if molino == "MC1":
#             capacidad = tolvas_MC1[material]["capacidad"]
#             print("\nEstado actual MC1:")
#             print(f"Niveles MC1: {redondear_diccionario(niveles_MC1)}")

#         elif molino == "MC2":
#             capacidad = tolvas_MC2[material]["capacidad"]
#         else:  # MC3
#             capacidad = tolvas_MC3[material]["capacidad"]

#         nivel_objetivo = min(cantidad_necesaria, capacidad * rendimiento)

#         if niveles[material] < nivel_objetivo:
#             materias_faltantes.append(material)

#     return materias_faltantes


def todas_tolvas_llenas():
    rendimiento_MC1_MC2 = 0.8
    rendimiento_MC3 = 0.5

    if any(nivel < tolvas_MC1[material]['capacidad'] * rendimiento_MC1_MC2 for material, nivel in niveles_MC1.items()):
        return False
    if any(nivel < tolvas_MC2[material]['capacidad'] * rendimiento_MC1_MC2 for material, nivel in niveles_MC2.items()):
        return False
    if any(nivel < tolvas_MC3[material]['capacidad'] * rendimiento_MC3 for material, nivel in niveles_MC3.items()):
        return False
    return True


def alimentar_materiales_faltantes():
    print("\nIntentando alimentar materiales faltantes...")
    for molino in ["MC1", "MC2", "MC3"]:
        if molino == "MC1":
            niveles = niveles_MC1
            tipo_produccion = "P30"  # Asumimos P30 como default
        elif molino == "MC2":
            niveles = niveles_MC2
            tipo_produccion = "P16"  # Asumimos P16 como default
        else:  # MC3
            niveles = niveles_MC3
            tipo_produccion = "P30"

        necesidades = calcular_necesidades(molino, tipo_produccion)
        for material, cantidad_necesaria in necesidades.items():
            if niveles[material] < cantidad_necesaria:
                print(f"Intentando alimentar {material} faltante en {molino}")
                cantidad_a_alimentar = cantidad_necesaria - niveles[material]
                intentos = 0
                while niveles[material] < cantidad_necesaria and intentos < 3:
                    cantidad_alimentada = intentar_alimentar(molino, material, cantidad_a_alimentar)
                    if cantidad_alimentada > 0:
                        niveles[material] += cantidad_alimentada
                        cantidad_a_alimentar -= cantidad_alimentada
                    else:
                        print(f"No se pudo alimentar {material} en {molino}. Intentando liberar rutas...")
                        liberar_rutas_alimentacion(molino, material)
                    intentos += 1

                if niveles[material] < cantidad_necesaria:
                    print(f"ADVERTENCIA: No se pudo alimentar completamente {material} en {molino}")


def verificar_materiales_faltantes():
    print("\nVerificando materiales faltantes...")
    for molino in ["MC1", "MC2", "MC3"]:
        if molino == "MC1":
            niveles = niveles_MC1
            print("\nEstado actual después de la alimentación MC1:")
            print(f"Niveles MC1: {redondear_diccionario(niveles_MC1)}")

        elif molino == "MC2":
            niveles = niveles_MC2
        else:  # MC3
            niveles = niveles_MC3

        for material, nivel in niveles.items():
            if nivel == 0:
                mostrar_mensaje_parpadeante(f"¡ALERTA! Falta alimentar {material} en {molino}")

def mostrar_mensaje_parpadeante(mensaje):
    def parpadeo():
        for _ in range(5):  # Parpadea 5 veces
            print(f"\r\033[91m{mensaje}\033[0m", end="", flush=True)
            time.sleep(0.5)
            print("\r" + " " * len(mensaje), end="", flush=True)
            time.sleep(0.5)
        print(f"\r{mensaje}")

    threading.Thread(target=parpadeo).start()


def liberar_rutas_alimentacion(molino, material):
    global alimentaciones_actuales, alimentaciones_en_progreso

    print(f"Liberando rutas de alimentación para {material} en {molino}")

    if material == "Yeso":
        # Liberar rutas específicas para Yeso
        alimentaciones_actuales = {(m, mat) for m, mat in alimentaciones_actuales if mat != "Yeso"}
        for m in alimentaciones_en_progreso:
            alimentaciones_en_progreso[m] = {mat for mat in alimentaciones_en_progreso[m] if mat != "Yeso"}

    elif material == "Clinker_Silo_Blanco" and molino == "MC3":
        # Liberar rutas específicas para Clinker_Silo_Blanco en MC3
        alimentaciones_actuales = {(m, mat) for m, mat in alimentaciones_actuales if mat != "Clinker" and m != "MC3"}
        if "MC3" in alimentaciones_en_progreso:
            alimentaciones_en_progreso["MC3"] = {mat for mat in alimentaciones_en_progreso["MC3"] if mat != "Clinker"}

    print(f"Rutas liberadas. Estado actual:")
    print(f"Alimentaciones actuales: {alimentaciones_actuales}")
    print(f"Alimentaciones en progreso: {alimentaciones_en_progreso}")

def redondear_diccionario(d, decimales=2):
    return {k: round(v, decimales) if isinstance(v, (int, float)) else v for k, v in d.items()}

def procesar_cola_alimentacion():
    while True:
        molino, material, cantidad = cola_alimentacion.get()
        print(f"Intentando alimentar {material} en {molino}")
        cantidad_alimentada = intentar_alimentar(molino, material, cantidad)
        if cantidad_alimentada > 0:
            print(f"Se alimentaron {cantidad_alimentada:.2f}t de {material} en {molino}")
        else:
            print(f"No se pudo alimentar {material} en {molino}. Se volverá a intentar más tarde.")
            # Volver a poner en la cola si no se pudo alimentar
            cola_alimentacion.put((molino, material, cantidad))
        cola_alimentacion.task_done()


def main():
    global niveles_MC1, niveles_MC2, niveles_MC3, tipos_produccion_actual
    ciclo = 1

    # Iniciar un hilo separado para manejar la alimentación de materiales
    threading.Thread(target=procesar_cola_alimentacion, daemon=True).start()

    while True:
        print(f"\n--- Ciclo de optimización {ciclo} ---")

        for molino in ["MC1", "MC2", "MC3"]:
            if molino == "MC1":
                tipos_validos = ["P30", "P40"]
            elif molino == "MC2":
                tipos_validos = ["P10", "P16", "P20", "P30"]
            elif molino == "MC3":
                tipos_validos = ["P30"]

            print(f"\nOptimizando {molino}")
            print(f"Este molino puede producir: {', '.join(tipos_validos)}")
            tipo_produccion = input(f"Ingrese el tipo de producción para {molino} (o presione Enter para omitir): ").upper()

            if tipo_produccion:
                if tipo_produccion not in tipos_validos:
                    print(f"Error, Ingrese un tipo de producción válido para {molino}")
                    continue
                tipos_produccion_actual[molino] = tipo_produccion  # Actualizar el tipo de producción actual
                optimizar_alimentacion(molino, tipo_produccion)


        print("\nEstado actual de las alimentaciones:")
        for molino, materiales in alimentaciones_en_progreso.items():
            if materiales:
                print(f"{molino}: {', '.join(materiales)}")
            else:
                print(f"{molino}: Sin alimentaciones en progreso")

        # Esperar a que todos los temporizadores terminen
        while any(temporizadores_molinos.values()):
            time.sleep(1)

        print("\nTodas las alimentaciones en progreso han sido completadas.")

        # Intentar alimentar materiales faltantes
        alimentar_materiales_faltantes()

        print("\nEstado actual después de la alimentación:")
        print(f"Niveles MC1: {redondear_diccionario(niveles_MC1)}")
        print(f"Niveles MC2: {redondear_diccionario(niveles_MC2)}")
        print(f"Niveles MC3: {redondear_diccionario(niveles_MC3)}")


        # Verificar y mostrar advertencias de materiales faltantes
        verificar_materiales_faltantes()

        print("\nEstado actual después de la alimentación:")
        print(f"Niveles MC1: {redondear_diccionario(niveles_MC1)}")
        print(f"Niveles MC2: {redondear_diccionario(niveles_MC2)}")
        print(f"Niveles MC3: {redondear_diccionario(niveles_MC3)}")


        if todas_tolvas_llenas():
            print("Todas las tolvas están llenas. Finalizando la simulación.")
            break

        ciclo += 1

        continuar = input("¿Desea continuar con otro ciclo? (s/n): ").lower()
        if continuar != 's':
            break

    print("Simulación completada.")


if __name__ == "__main__":
    main()

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Cantidad solicitada: 0.93t

--- Seleccionando ruta para Yeso en MC3 ---
Verificando condiciones para MC3...
Se puede alimentar por L1
No se está alimentando Clinker a L3
Ruta seleccionada: Hacia MC3 por MC1
Verificando restricciones...
No se puede alimentar debido a restricciones: No se puede alimentar Yeso mientras se alimenta Puzolana
No se pudo alimentar Yeso en MC3. Se volverá a intentar más tarde.
Intentando alimentar Yeso en MC3

--- Intentando alimentar Yeso a MC3 ---
Cantidad solicitada: 0.93t

--- Seleccionando ruta para Yeso en MC3 ---
Verificando condiciones para MC3...
Se puede alimentar por L1
No se está alimentando Clinker a L3
Ruta seleccionada: Hacia MC3 por MC1
Verificando restricciones...
No se puede alimentar debido a restricciones: No se puede alimentar Yeso mientras se alimenta Puzolana
No se pudo alimentar Yeso en MC3. Se volverá a intentar más tarde.
Intentando alimentar Yeso en MC3

--- Intentand

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-ac72df7edf1d>", line 1123, in <cell line: 1122>
    main()
  File "<ipython-input-1-ac72df7edf1d>", line 1115, in main
    continuar = input("¿Desea continuar con otro ciclo? (s/n): ").lower()
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py", line 851, in raw_input
    return self._input_request(str(prompt),
  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelbase.py", line 895, in _input_request
    raise KeyboardInterrupt("Interrupted by user") from None
KeyboardInterrupt: Interrupted by user

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_trac

TypeError: object of type 'NoneType' has no len()